**My first data set analysis. **

In [1]:
#Import all packages

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Reading file and performing cleanup

players = pd.read_csv("../input/wta/players.csv",  encoding='latin', index_col=0)
# Top column is misaligned.
players.index.name = 'ID'
players.columns = ['First' , 'Last', 'Hand', 'DOB', 'Country']
# Parse date data to dates.
players = players.assign(DOB=pd.to_datetime(players['DOB'],format='%Y%m%d'))
# Handidness is reported as U if unknown; set np.nan instead.
players = players.assign(Hand=players['Hand'].replace('U', np.nan))
# Included year of birth  column for graph 
players['YOB'] = players['DOB'].apply(lambda b: b.year)

**Left and Right hand players  over the years**

In [3]:
players.groupby(['YOB', 'Hand']) \
 .size().unstack('Hand').replace(np.NaN, 0).plot(title="Players' hand over the years",figsize=(20,10))

**Top 10 countries representing Womens Tennis**

In [4]:
players.groupby('Country') \
            .size() \
            .sort_values(ascending=False)[:10] \
            .plot.bar(title="Top 10 countries with the most player", 
                  figsize=(20,10),width=0.30)

**Matches Data set**

Top 10 winners

In [5]:
#### Matches:
matches = pd.read_csv("../input/wta/matches.csv", encoding='latin1', index_col=0)
## Most Wins - Top 10. : 
#print('Most Wins')
#print(matches.groupby(['winner_name']).winner_name.count().nlargest(10))
## Most Wins - Top 10. : 
top10Winners = matches.groupby(['winner_name']).winner_name.count().nlargest(10)
# Get individual match win breakup

grouped = matches.groupby(['winner_name','surface'])['winner_ioc'].count()
groupedDataset  = pd.DataFrame(grouped)
p1 = pd.DataFrame(groupedDataset.pivot_table(index="winner_name",columns = "surface", aggfunc='sum',fill_value=0).reset_index())
p1.columns = p1.columns.droplevel(0)
p1.columns.values[0] = 'Winner Name'
p1['Total'] = p1['Carpet'] + p1['Clay']+p1['Grass']+ p1['Hard']
top10Breakup = p1.drop(['6-2 6-3'] , axis=1)
#print(top10Breakup.nlargest(10,'Total'))
top10Winners.plot.bar(figsize=(15,8),title="Top 10 players with most wins",width=0.25,color='burlywood')
top10Breakup.nlargest(10,'Total').plot.bar(title='A breakup of the wins',figsize=(15,8),x="Winner Name", y=["Carpet",'Clay','Hard','Grass'],stacked=True,color=['gray','firebrick', 'steelblue', 'forestgreen'],alpha=0.9, width=0.25);

In [ ]:
rgbl = [125,130,18]
matches.groupby(['winner_ioc']).winner_ioc.count().nlargest(10).plot.bar(figsize=(15,8),title="Top 10 countries with most wins",width=0.25,color=np.random.shuffle(rgbl))


In [ ]:
rgbl = [125,130,18]
matches.groupby(['loser_ioc']).winner_ioc.count().nlargest(10).plot.bar(figsize=(15,8),title="Top 10 countries with most loss",width=0.25,color='black',alpha=0.6)


**Championship winners distibution based on age:**

In [ ]:
# Distubition of trophy winners based on age
matchesWon = matches[(matches['round'] == 'F') & 
(matches['tourney_name'] =='US Open')| \
(matches['tourney_name'] =='French Open')|(matches['tourney_name'] =='Wimbledon')| \
                     (matches['tourney_name'] =='Australian Open')]
matchesWon.winner_age.plot(kind='hist',color='gold',label='Age',bins = 150 \
                           ,linewidth=0.01,grid=True,figsize = (12,10))
plt.xlabel('Age')              # label = name of label
plt.ylabel('No of Wins')
plt.suptitle('Number of Grandslams wins based on Age of the player', x=0.5, y=.9, ha='center', fontsize='xx-large')


**Grand Slam Tournament Winners over the years:**

In [6]:
ausWinners = matches[(matches['tourney_name'] =='Australian Open') & (matches['round'] == "F")]
frenchWinners = matches[(matches['tourney_name'] =='French Open') & (matches['round'] == "F")]
wimbeldonWinners = matches[(matches['tourney_name'] =='Wimbledon') & (matches['round'] == "F")]
usWinners = matches[(matches['tourney_name'] =='US Open') & (matches['round'] == "F")]
#color=['gray','firebrick', 'steelblue', 'forestgreen']
pd.DataFrame(ausWinners.groupby(['winner_name'])['round'].count().nlargest(10)).plot.bar(figsize=(15,8),title="Top 10 players with most Aus open titles",width=0.25,color='royalblue')
pd.DataFrame(frenchWinners.groupby(['winner_name'])['round'].count().nlargest(10)).plot.bar(figsize=(15,8),title="Top 10 players with most French Open open titles",width=0.25,color='darkorange')
pd.DataFrame(wimbeldonWinners.groupby(['winner_name'])['round'].count().nlargest(10)).plot.bar(figsize=(15,8),title="Top 10 players with most Wimbledon open titles",width=0.25,color='lawngreen')
pd.DataFrame(usWinners.groupby(['winner_name'])['round'].count().nlargest(10)).plot.bar(figsize=(15,8),title="Top 10 players with most US open titles",width=0.25,color='steelblue')


**Championship winnablity:**
  The distibution shows the number of players  along with the age who  won a tournament as a wild card entry.

In [7]:
finalMatchesWildCard = matches[(matches['round'] == 'F') & (matches['winner_entry'] == 'WC') ]
finalMatchesWildCard.winner_age.plot(kind = 'hist', color = 'g',linestyle = '-',label = 'winner_age',linewidth=1,alpha = 0.5,grid = True
                                     ,edgecolor='black',figsize=(12,8))

**Performace of Wild Card entrants** : 

    It is observed that wild card entrants mostly pass through round of 32. Few wild card entrants have even won the tournament

In [ ]:
wildCard = matches[(matches['winner_entry'] == 'WC')]
wildCard.groupby(['winner_entry', 'round']) \
 .size().unstack('winner_entry').replace(np.NaN, 0).plot(title="Wild Card entries at events",figsize=(20,10))
    
    # random matchesWon.plot.hexbin(title="Top 10 countries with the most player", x='winner_age' ,y='winner_rank_points',                  figsize=(20,10))